**Oleobrás em busca do melhor lugar para um novo poço!**

Nesse projeto vamos desenvolver um modelo para encontrar o melhor lugar para um novo poço da empresa de mineração Oleobrás. Iremos coletar os parâmetros de poços de petróleo na região selecionada: a qualidade de petróleo e o volume de reservas, construir um modelo para predizer o volume de reservas nos novos poços, Escolher os poços de petróleo que têm os maiores valores estimados e por fim, escolher a região com o maior lucro total para os poços de petróleo selecionados.



Primeiro passo será importar as bibliotecas necessárias.

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error


Abaixo li os arquivos com os dados de cada região:

In [2]:
df_zero = pd.read_csv('/datasets/geo_data_0.csv')
df_one = pd.read_csv('/datasets/geo_data_1.csv')
df_two = pd.read_csv('/datasets/geo_data_2.csv')

A função resumo foi criada para facilitar a apresentação dos dados relevantes de cada tabela. 

In [3]:
def resumo(df):
    print(df)
    print(df.info())
    print(df.describe())

In [4]:
resumo(df_zero)

          id        f0        f1        f2     product
0      txEyH  0.705745 -0.497823  1.221170  105.280062
1      2acmU  1.334711 -0.340164  4.365080   73.037750
2      409Wp  1.022732  0.151990  1.419926   85.265647
3      iJLyR -0.032172  0.139033  2.978566  168.620776
4      Xdl7t  1.988431  0.155413  4.751769  154.036647
...      ...       ...       ...       ...         ...
99995  DLsed  0.971957  0.370953  6.075346  110.744026
99996  QKivN  1.392429 -0.382606  1.273912  122.346843
99997  3rnvd  1.029585  0.018787 -1.348308   64.375443
99998  7kl59  0.998163 -0.528582  1.583869   74.040764
99999  1CWhH  1.764754 -0.266417  5.722849  149.633246

[100000 rows x 5 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2  

In [5]:
resumo(df_one)

          id         f0         f1        f2     product
0      kBEdx -15.001348  -8.276000 -0.005876    3.179103
1      62mP7  14.272088  -3.475083  0.999183   26.953261
2      vyE1P   6.263187  -5.948386  5.001160  134.766305
3      KcrkZ -13.081196 -11.506057  4.999415  137.945408
4      AHL4O  12.702195  -8.147433  5.004363  134.766305
...      ...        ...        ...       ...         ...
99995  QywKC   9.535637  -6.878139  1.998296   53.906522
99996  ptvty -10.160631 -12.558096  5.005581  137.945408
99997  09gWa  -7.378891  -3.084104  4.998651  137.945408
99998  rqwUm   0.665714  -6.152593  1.000146   30.132364
99999  relB0  -3.426139  -7.794274 -0.003299    3.179103

[100000 rows x 5 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non

In [6]:
resumo(df_two)

          id        f0        f1        f2     product
0      fwXo0 -1.146987  0.963328 -0.828965   27.758673
1      WJtFt  0.262778  0.269839 -2.530187   56.069697
2      ovLUW  0.194587  0.289035 -5.586433   62.871910
3      q6cA6  2.236060 -0.553760  0.930038  114.572842
4      WPMUX -0.515993  1.716266  5.899011  149.600746
...      ...       ...       ...       ...         ...
99995  4GxBu -1.777037  1.125220  6.263374  172.327046
99996  YKFjq -1.261523 -0.894828  2.524545  138.748846
99997  tKPY3 -1.199934 -2.957637  5.219411  157.080080
99998  nmxp2 -2.419896  2.417221 -5.548444   51.795253
99999  V9kWn -2.551421 -2.025625  6.090891  102.775767

[100000 rows x 5 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2  

Analisando as tabelas, não foi preciso fazer nenhuma adequação. Não havia dados duplicados, nem nulos. Portanto, segui com o estudo.

Dividi as 3 tabelas em target (objetivo) e features (características). Target representa a coluna 'product'(volume de reservas no poço de petróleo (milhares de barris)) e features as outras com exceção das colunas:'product'e 'id'. A coluna 'id' não acrescentaria nenhuma informação relevante ao modelo.

In [7]:
target_zero = df_zero['product']
features_zero = df_zero.drop(['product','id'], axis=1)

In [8]:
target_one = df_one['product']
features_one = df_one.drop(['product','id'], axis=1)

In [9]:
target_two = df_two['product']
features_two = df_two.drop(['product','id'], axis=1)

Para nosso modelo funcionar precisamos ter além de um conjunto para treinar o modelo, um conjunto para validá-lo. Por isso, nos próximos códigos dividi os dados da seguinte forma: 75% conjunto de teste e 25% conjunto de validação.

In [10]:
features_train_zero, features_valid_zero, target_train_zero, target_valid_zero = train_test_split(
    features_zero, target_zero, test_size=0.25, random_state=12345
)

In [11]:
features_train_one, features_valid_one, target_train_one, target_valid_one = train_test_split(
    features_one, target_one, test_size=0.25, random_state=12345
)

In [12]:
features_train_two, features_valid_two, target_train_two, target_valid_two = train_test_split(
    features_two, target_two, test_size=0.25, random_state=12345
)

O projeto exige que o nosso modelo seja estimado somente através de regressão linear. Portanto, treinei o modelo de regressão linear para cada região e com isso obtive suas predições. 

In [13]:
model_zero = LinearRegression()
model_zero.fit(features_train_zero,target_train_zero)
predictions_valid_zero = model_zero.predict(features_valid_zero)


In [14]:
model_one = LinearRegression()
model_one.fit(features_train_one,target_train_one)
predictions_valid_one = model_one.predict(features_valid_one)

In [15]:
model_two = LinearRegression()
model_two.fit(features_train_two,target_train_two)
predictions_valid_two = model_two.predict(features_valid_two)

Com as predições de cada dataframe calculadas, posso gerar  algumas métricas muito importantes quando estamos estudando regressões. Abaixo criei uma função que facilita esses cálculos para cada região. Iremos descobrir o REQM, erro médio absoluto, o volume médio e o R2 de cada região.    

In [16]:
def linear_reg(features_train,target_train,features_valid,target_valid,predictions_valid):

    
    REQM = mean_squared_error(target_valid, predictions_valid)**0.5
    print("O REQM do modelo de regressão linear no conjunto de validação:", REQM)
    
    EMA = mean_absolute_error(target_valid,predictions_valid)
    print("O Erro médio Absoluto do modelo de regressão linear no conjunto de validação:", EMA)
    
    print("O volume médio da região no conjunto de predição:", predictions_valid.mean())
    print("O volume médio da região no conjunto de validação :", target_valid.mean())
    
    r2 = r2_score(target_valid,predictions_valid)
    print('R2_zero =', r2)  

In [17]:
linear_reg(features_train_zero,target_train_zero,features_valid_zero,target_valid_zero,predictions_valid_zero)

O REQM do modelo de regressão linear no conjunto de validação: 37.5794217150813
O Erro médio Absoluto do modelo de regressão linear no conjunto de validação: 30.919600777151313
O volume médio da região no conjunto de predição: 92.59256778438035
O volume médio da região no conjunto de validação : 92.07859674082927
R2_zero = 0.27994321524487786


In [18]:
linear_reg(features_train_one,target_train_one,features_valid_one,target_valid_one,predictions_valid_one)

O REQM do modelo de regressão linear no conjunto de validação: 0.893099286775617
O Erro médio Absoluto do modelo de regressão linear no conjunto de validação: 0.7187662442124758
O volume médio da região no conjunto de predição: 68.728546895446
O volume médio da região no conjunto de validação : 68.72313602435997
R2_zero = 0.9996233978805127


In [19]:
linear_reg(features_train_two,target_train_two,features_valid_two,target_valid_two,predictions_valid_two)

O REQM do modelo de regressão linear no conjunto de validação: 40.02970873393434
O Erro médio Absoluto do modelo de regressão linear no conjunto de validação: 32.792652105481814
O volume médio da região no conjunto de predição: 94.96504596800489
O volume médio da região no conjunto de validação : 94.88423280885438
R2_zero = 0.20524758386040443


Percebemos que a região 2 (df_one) tem o menor valor de REQM e EMA, sinalizando que suas predições têm um nível menor de erro em relação as outras duas regiões. Essa região também tem o menor R2 (Coeficiente de determinação) assim, sendo o modelo que melhor prevê as repostas. Essas métricas mostram que o modelo de regressão linear é o mais confiável. Contudo, em relação à média, as regiões 1 e 3 possuem consideravelmente um volume maior de barris de petróleo. A região 3, por exemplo, tem em média 94 mil barris de petróleo enquanto a regiao 2, apenas 68. 

O projeto demanda que criemos uma função para calcular o lucro e que calculemos o volume mínimo de reservas suficientes para desenvolver um novo poço sem prejuízo.   

In [20]:
custo = 100000000
receita_per_mil_barris = 4500
lucro_min = 0

In [21]:
volume_min = (lucro_min+custo)/receita_per_mil_barris
print('Volume Minimo por região:',round(volume_min,2))

Volume Minimo por região: 22222.22


In [22]:
custo_per_bar = 100000000/200


In [23]:
volume_min = (lucro_min+custo_per_bar)/receita_per_mil_barris
print('Volume Minimo por poço:',round(volume_min,2))

Volume Minimo por poço: 111.11


O volume mínimo de reservas para desenvolver o poço sem prejuízos é de 111.11 milhares de barris. Esse número é superior à média encontrada nas 3 regiões analisadas. Como estamos buscando 200 poços, o volume mínimo por região é de 222.22. Abaixo foi definida a função lucro que será de extrema relevância para a continuação do estudo.    

In [24]:
def lucro(custo,receita,total_volume):
    lucro = (total_volume*receita)-custo
    return lucro
    

Para os próximos passos foi necessário transformar os dados previstos pelo modelo no formato Series. 

In [25]:
predictions_valid_zero = pd.Series(predictions_valid_zero)
predictions_valid_one = pd.Series(predictions_valid_one)
predictions_valid_two = pd.Series(predictions_valid_two)

Conforme foi solicitado no projeto, criei uma função que calcula o lucro total colhendo 500 pontos aleatórios e escolhendo os 200 melhores de acordo com as predições do modelo de regressão linear para cada região.   

In [26]:
def calculate_profit(target,predictions, num_wells=200, custo=100000000,receita=4500):
    state = np.random.RandomState(54321)
    sample = predictions.sample(n=500,replace=True,random_state = state)
    index = sample.sort_values(ascending=False)[:num_wells].index
    total_volume = target.reset_index(drop=True)[index].sum()
    total_profit = lucro(custo,receita,total_volume)
    
    
    print(f"Volume de reservas: {total_volume:.2f} milhares de barris")
    print(f"Lucro total: ${total_profit:.2f}")

In [27]:
calculate_profit(target_valid_zero,predictions_valid_zero)

Volume de reservas: 23223.67 milhares de barris
Lucro total: $4506494.44


In [28]:
calculate_profit(target_valid_one,predictions_valid_one)

Volume de reservas: 23172.89 milhares de barris
Lucro total: $4278020.28


In [29]:
calculate_profit(target_valid_zero,predictions_valid_zero)

Volume de reservas: 23223.67 milhares de barris
Lucro total: $4506494.44


A região 1 e 3 foram as com maiores lucros e reservas de barris de petróleo. Se analisássemos só dessa forma tenderíamos a não escolher a região 2 como possível opção para o investimento. Entretanto, não estamos levando em conta a probabilidade de escolhermos poços onde teríamos prejuízo. Por isso, abaixo utilizamos a técnica bootstrapping com 1000 amostras para encontrar a distribuição de lucros. Encontramos o lucro médio, intervalo de confiança de 95% e o risco de prejuízo.   

In [30]:
def boot(target,predictions, num_wells=200, custo=100000000,receita=4500):
    state = np.random.RandomState(54321)
    values = []
    for i in range(1000):
        subsample = predictions.sample(n=500, replace=True, random_state=state)
        index = subsample.sort_values(ascending=False)[:num_wells].index
        total_volume = target.reset_index(drop=True)[index].sum()
        profit = lucro(custo,receita,total_volume) 
        values.append(profit)
       

    
    values = pd.Series(values)    

    lower = values.quantile(0.025) 
    upper = values.quantile(0.975)
    mean = values.mean()
    preju_per = values[values<0].count()/len(values)*100

    print('Limite inferior: $',round(lower,2))
    print('Limite Superior: $',round(upper,2))
    print('Lucro Médio da Região: $',round(mean,2))
    print('Risco de Prejuizo na região:',preju_per,'%')

nao entendi essa parte, pode explicar melhor?

In [31]:
boot(target_valid_zero,predictions_valid_zero)

Limite inferior: $ -1108640.03
Limite Superior: $ 9033598.34
Lucro Médio da Região: $ 3920829.09
Risco de Prejuizo na região: 6.4 %


In [32]:
boot(target_valid_one,predictions_valid_one)

Limite inferior: $ 338267.29
Limite Superior: $ 8465670.69
Lucro Médio da Região: $ 4373341.52
Risco de Prejuizo na região: 1.5 %


In [33]:
boot(target_valid_zero,predictions_valid_zero)

Limite inferior: $ -1108640.03
Limite Superior: $ 9033598.34
Lucro Médio da Região: $ 3920829.09
Risco de Prejuizo na região: 6.4 %


Analisando a distribuição dos lucros, vimos que apenas a região 2 no intervalo de 95% de confiança não tem prejuízos. Também somente ela tem risco de perdas abaixo de 2,5%. As regiões 1 e 3 por terem riscos de prejuízos maiores do que 2,5% foram automaticamente excluídas. Por ter menor risco de perdas e ainda pior cima um lucro médio maior a região 2 (df_one)) foi escolhida para o novo poço de petróleo. Também vale destacar que as predições da regressão linear de df_one obtiveram os melhores resultados.